In [ ]:
pip install PyPDF2

In [ ]:
pip install -U langchain-community

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
import os
os.environ["OPENAI_API_KEY"]=""

In [ ]:
pdfreader = PdfReader("Rohith_CV.pdf")

In [ ]:
from typing_extensions import Concatenate
raw_data=''
for i, page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
    raw_data+=content

In [ ]:
raw_data

In [ ]:
text_spiltter = CharacterTextSplitter(        # charactertextsplitter used to split the large chunks of text(like pdf content) into smaller overlapping segments,which makes easier to pass to LLM for que-ans or summarize. it splits text based on character and specified separator.
    separator='\n',
    chunk_size=800,  #each chunk of text is 800 character long
    chunk_overlap=200, #each chunk will overlap the previous one by 200 character
    length_function=len,

)

In [ ]:
text=text_spiltter.split_text(raw_data)

In [ ]:
len(text)

to count tokens instead of length (i'm doing this on my own)

In [ ]:
import tiktoken

# Choose tokenizer – for GPT-3.5/GPT-4 use 'cl100k_base'

encoding = tiktoken.get_encoding('cl100k_base')

In [ ]:
def count_tokens(text:str)->int:
  return len(encoding.encode(text))

or def count_tokens(raw_data:str)->int:
    return len(encoding.encode(raw_data))  
this also gives me same result.

In [ ]:
text_spiltter = CharacterTextSplitter(
    separator='/n',
    chunk_size=800,
    chunk_overlap=200,
    length_function=count_tokens
)

In [ ]:
text = text_spiltter.split_text(raw_data)

In [ ]:
count_tokens(text)

You're passing text — which is a list of strings (the chunks) — to count_tokens(), which expects a single string.

ToFix:

You need to either:

Count tokens for each chunk individually, or

Join all chunks back into one string before counting

In [ ]:
# count token for each chunck.
for i, chunk in enumerate(pdfreader.pages):
  print(f"chunk {i+1} - Tokens: {count_tokens(chunk)}")

you're passing a PageObject from PyPDF2 directly to your count_tokens() function — but count_tokens() expects a string, not a page object.

tofix

we need to extract the text from pages first.


why this works.

pdfreader.pages[i] returns a PageObject

page.extract_text() returns a string

count_tokens() expects a string, not a PageObject

In [ ]:
# count token for each page.
for i, page in enumerate(pdfreader.pages):
  text=page.extract_text()
  if text:
    print(f"Page {i+1} - Tokens: {count_tokens(text)}")
  else:
    print(f"Page {i+1} - No extractable text")

In [ ]:
print("Total tokens :",count_tokens(" ".join(text)))

# f stands for f-string short for formatted string literal
# e.g

name = 'rohith'
age = 25

print(f"My name is {name} and my age is {age}")

In [ ]:
embedding = OpenAIEmbeddings()

here api is a kind of password that allows our code to securely access services provided by our external api - in this case OpenAI API

OpenAI API key is a unique string (it look like this sk-...) that links API usage to our OpenAI a/c

to use in code,
import os
os.environ["OPENAI_API_KEY"]="sk-..........."

OpenAI API is paid, pay for what i have used-usage is measured in tokens(words or characters).
In my openai a/c is there is $5 in free trial means it is for 5$ dollars if not it shows 0.00 then we can't access without paying.

In [ ]:
doc_search = FAISS.from_texts(text,embedding)  #text is converting into embedding and stored in doc_search

In [ ]:
doc_search

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(),chain_type='stuff')

In [ ]:
query = " "
docs = doc_search.similarity_search(query)
chain.run(input_documents=docs, questions=query)